# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [15]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [88]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [42]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [312]:
document = ET.parse( './data/mondial_database.xml' )

In [313]:
#1. 10 countries with the lowest infant mortality rates
import pandas as pd

list = []
for element in document.getiterator('country'):
    if(element.find('infant_mortality') != None):
        name = element.find('name').text
        infant_mortality = float(element.find('infant_mortality').text)

    tuple = {'country' : name, 'infant_mortality' : infant_mortality}
    list.append(tuple)

sorted_list = sorted(list, key=lambda x: x['infant_mortality'])

pd.DataFrame(sorted_list).head(10)

,country,infant_mortality
0,Monaco,1.81
1,Japan,2.13
2,Norway,2.48
3,Bermuda,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


In [314]:
#2. 10 cities with the largest population
import pandas as pd

list = []
for element in document.getiterator('city'):
    city_name = element.find('name').text
    for subelement in element.getiterator('population'):
        year = int(subelement.attrib['year'])
        pop = float(subelement.text)
        list.append({'city':city_name,'year':year,'pop':pop})

df = pd.DataFrame(list)
df.sort_values(by='year', ascending=False).groupby(df.city).first().sort_values(by='pop',ascending=False).head(10)

,city,pop,year
city,,,
Shanghai,Shanghai,22315474,2010
Istanbul,Istanbul,13710512,2012
Mumbai,Mumbai,12442373,2011
Moskva,Moskva,11979529,2013
Beijing,Beijing,11716620,2010
São Paulo,São Paulo,11152344,2010
Tianjin,Tianjin,11090314,2010
Guangzhou,Guangzhou,11071424,2010
Delhi,Delhi,11034555,2011


In [315]:
#3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
import pandas as pd
import numpy as np

list = []
for element in document.getiterator('country'):
    # FInd latest population
    max_pop_year = 0
    max_pop = 0
    
    for popObj in element.findall('population'):
        if(int(popObj.attrib['year']) > max_pop_year):
            max_pop_year = int(popObj.attrib['year'])
            max_pop = float(popObj.text)
    
    for subelement1 in element.getiterator('ethnicgroup'):
        group_name = subelement1.text
        group_pop = (float(subelement1.attrib['percentage']) / 100 ) * max_pop
        list.append({'ethinic_group':group_name, 'population':group_pop })

df = pd.DataFrame(list)    
df.groupby(df.ethinic_group).aggregate(np.sum).sort_values(by='population', ascending=False).head(10)

,population
ethinic_group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [316]:
#4.name and country of a) longest river, b) largest lake and c) airport at highest elevation
import pandas as pd
river = []
lake = []
airport = []

for element in document.getiterator('river'):
    lengthEl = element.find('length')
    if(lengthEl == None):
        length = 0
    else:
        length = lengthEl.text
    river.append({'name':element.find('name').text, 'country':element.attrib['country'], 'length':float(length)})

for element in document.getiterator('lake'):
    depthEl = element.find('depth')
    if(depthEl == None):
        depth = 0
    else:
        depth = depthEl.text
    lake.append({'name':element.find('name').text, 'country':element.attrib['country'], 'depth':float(depth)})

for element in document.getiterator('airport'):
    elevationEl = element.find('elevation')
    if(elevationEl == None):
        elevation = 0
    else:
        if(elevationEl.text == None):
            elevation = 0
        else:
            elevation = elevationEl.text
    
    airport.append({'name':element.find('name').text, 'country':element.attrib['country'], 'elevation':float(elevation)})

df_river = pd.DataFrame(river)
df_lake = pd.DataFrame(lake)
df_airport = pd.DataFrame(airport)

print df_river.sort_values(by='length',ascending=False).head(1)
print df_lake.sort_values(by='depth',ascending=False).head(1)
print df_airport.sort_values(by='elevation',ascending=False).head(1)
    

      country  length      name
174  CO BR PE    6448  Amazonas
   country  depth          name
43       R   1637  Ozero Baikal
   country  elevation          name
80     BOL       4063  El Alto Intl
